In [31]:
import numpy as np
import torch

In [32]:
from config import Config
from data import Data
from channel import Channel

In [33]:
config = Config(
                N_transmit_antenna=128,
                N_active_antenna=8,
                N_receive_antenna=32,
                block_length=20,
                channel_length=3,
                channel_truncation='tail',
                alphabet='OOK',
                batch=1,
                generator_mode='segmented',
                iterations=50
)
data = Data(config)
channel = Channel(config)

In [34]:
H = channel.generate_channel()
g = torch.linalg.svdvals(H).cpu().numpy()**2

In [35]:
def waterfilling(gain, sigma2, eps=1e-10):
    P = config.Na / config.Nt
    N = len(gain)
    L = np.min(sigma2 / gain)
    U = (P + np.sum(sigma2 / gain))/N
    p = np.zeros(N)
    while np.abs(U - L) > eps:
        mu = (U + L) / 2
        p = mu - sigma2 / gain
        p[p<0] = 0
        if P > np.sum(p):
            U = mu
        else:
            L = mu
    return p

In [36]:
SNR = 10
sigma2 = config.Na / config.Nt / SNR
p = waterfilling(g, sigma2)

In [69]:
H = channel.generate_channel()
U, s, Vh = torch.linalg.svd(H, full_matrices=False)
x = data.generate_message()[0]

In [70]:
xt = (U @ Vh @ x)
xt.var()

tensor(0.0617, device='cuda:0')

In [71]:
config.Na / config.Nt

0.0625

In [72]:
z = s.view(-1, 1) * xt
z.var()

tensor(0.2389, device='cuda:0')

In [41]:
s.view(-1, config.Na)[:, 0].var()

tensor(0.3889, device='cuda:0')

In [42]:
config.Na / config.Nr

0.25

In [43]:
config.Nr * (config.Lin + config.Lh - 1)

704

In [44]:
x.std()

tensor(0.2421, device='cuda:0')

In [45]:
Vh.size()

torch.Size([704, 2560])

In [46]:
C = []

In [47]:
C.append([1, 2])

In [48]:
C

[[1, 2]]

In [49]:
np.abs(np.random.randn(10, 1)) 

array([[0.52750369],
       [0.23612164],
       [1.44740719],
       [0.22836174],
       [1.33596437],
       [0.71507288],
       [1.14509102],
       [1.17415583],
       [0.14004429],
       [1.29586171]])

In [50]:
def doWF(vtChannels: np.ndarray,
         dPt: float,
         noiseVar: float = 1.0,
         Es: float = 1.0):
    """
    Performs the Waterfilling algorithm and returns the optimum power and
    water level.

    Parameters
    ----------
    vtChannels : np.ndarray
        Numpy array with the channel POWER gains (power of the parallel
        AWGN channels).
    dPt : float
        Total available power.
    noiseVar : float
        Noise variance (power in linear scale).
    Es : float
        Symbol energy (in linear scale).

    Returns
    -------
    (vtOptP, mu) : (np.ndarray, float)
        A tuple with vtOptP and mu, where vtOptP are the optimum powers,
        while mu is the water level.
    """
    # Sort Channels (descending order)
    vtChannelsSortIndexes = np.argsort(vtChannels)[::-1]
    vtChannelsSorted = vtChannels[vtChannelsSortIndexes]
    assert isinstance(vtChannelsSorted, np.ndarray)

    # xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
    # Calculates the water level that touches the worst channel (the higher
    # one) and therefore transmits zero power in this worst channel. After
    # that, calculates the power in each channel (the vector 'Ps') for this
    # water level. If the sum of all of these powers in 'Ps' is less then
    # the total available power, then all we need to do is divide the
    # remaining power equally among all the channels (increase the water
    # level). On the other hand, if the sum of all of these powers in 'Ps'
    # is greater then the total available power then we remove the worst
    # channel and repeat the process.
    # xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
    # Calculates minimum water-level $\mu$ required to use all channels
    dNChannels = vtChannels.size
    dRemoveChannels = 0

    minMu = float(noiseVar) / (
        Es * vtChannelsSorted[dNChannels - dRemoveChannels - 1])
    Ps = (minMu - float(noiseVar) /
          (Es * vtChannelsSorted[np.arange(0, dNChannels - dRemoveChannels)]))

    # Ps should be a numpy array
    assert isinstance(Ps, np.ndarray)

    while (sum(Ps) > dPt) and (dRemoveChannels < dNChannels):
        dRemoveChannels += 1
        minMu = float(noiseVar) / (
            Es * vtChannelsSorted[dNChannels - dRemoveChannels - 1])
        Ps = (
            minMu - float(noiseVar) /
            (Es * vtChannelsSorted[np.arange(0, dNChannels - dRemoveChannels)])
        )

    # Distributes the remaining power among the all the remaining channels
    dPdiff = dPt - np.sum(Ps)
    vtOptPaux = dPdiff / (dNChannels - dRemoveChannels) + Ps

    # Put optimum power in the original channel order
    vtOptP = np.zeros([
        vtChannels.size,
    ])
    vtOptP[vtChannelsSortIndexes[np.arange(0, dNChannels -
                                           dRemoveChannels)]] = vtOptPaux
    mu = vtOptPaux[0] + float(noiseVar) / vtChannelsSorted[0]

    return vtOptP, mu

In [51]:
p = doWF(g, config.Na/config.Nt, sigma2)[0]

In [52]:
np.sum(p)

0.06250000104409992

In [53]:
config.Na / config.Nt

0.0625

In [54]:
def water_filling(gain: np.ndarray, sigma2: float):
    P = config.Na / config.Nt
    NChannels = gain.size
    RemoveChannels = 0

    minMu = sigma2 / gain[NChannels - RemoveChannels - 1]
    Ps = minMu - sigma2 / gain[np.arange(NChannels - RemoveChannels)]

    while (sum(Ps) > P) and (RemoveChannels < NChannels):
        RemoveChannels += 1
        minMu = sigma2 / gain[NChannels - RemoveChannels - 1]
        Ps = minMu - sigma2 / gain[np.arange(NChannels - RemoveChannels)]

    # Distributes the remaining power among the all the remaining channels
    Pdiff = P - np.sum(Ps)
    Paux = Pdiff / (NChannels - RemoveChannels) + Ps

    # Put optimum power in the original channel order
    Palloc = np.zeros(NChannels)
    Palloc[np.arange(NChannels - RemoveChannels)] = Paux

    return Palloc

In [55]:
p = water_filling(g, sigma2)

In [56]:
np.sum(p)

0.06250000104409992

In [57]:
p

array([5.49380085e-04, 5.46347641e-04, 5.41795511e-04, 5.40803303e-04,
       5.38204680e-04, 5.35926432e-04, 5.35213505e-04, 5.33025246e-04,
       5.31621685e-04, 5.29107987e-04, 5.24142582e-04, 5.18854416e-04,
       5.14234998e-04, 5.13221778e-04, 5.11309947e-04, 5.09601843e-04,
       5.08393103e-04, 5.06491226e-04, 5.05715434e-04, 5.03457268e-04,
       5.02325129e-04, 5.00620168e-04, 4.98759386e-04, 4.98170150e-04,
       4.94450447e-04, 4.91445477e-04, 4.89864207e-04, 4.89623053e-04,
       4.87916142e-04, 4.86709323e-04, 4.85454308e-04, 4.83508076e-04,
       4.81730996e-04, 4.79696348e-04, 4.78212169e-04, 4.75893990e-04,
       4.70991101e-04, 4.69708320e-04, 4.65531164e-04, 4.60729614e-04,
       4.53293411e-04, 4.49018407e-04, 4.42041812e-04, 4.40558972e-04,
       4.39147087e-04, 4.37321229e-04, 4.34300950e-04, 4.33114212e-04,
       4.32183995e-04, 4.31329507e-04, 4.30228567e-04, 4.29545325e-04,
       4.28885134e-04, 4.27502760e-04, 4.25622246e-04, 4.24533297e-04,
      

In [58]:
15.10050534 / 8

1.8875631675

In [59]:
g

array([9.242743  , 9.201478  , 9.140223  , 9.126979  , 9.092474  ,
       9.062438  , 9.05308   , 9.024475  , 9.006223  , 8.973718  ,
       8.910194  , 8.843523  , 8.786095  , 8.773598  , 8.750114  ,
       8.729239  , 8.714527  , 8.691479  , 8.682113  , 8.654963  ,
       8.641415  , 8.621092  , 8.599021  , 8.592055  , 8.548343  ,
       8.513352  , 8.495055  , 8.492271  , 8.472621  , 8.458782  ,
       8.444439  , 8.422292  , 8.402171  , 8.3792515 , 8.362612  ,
       8.336753  , 8.282586  , 8.26853   , 8.223087  , 8.171465  ,
       8.092784  , 8.048233  , 7.9765725 , 7.961506  , 7.9472127 ,
       7.9288044 , 7.898541  , 7.8867126 , 7.8774657 , 7.868991  ,
       7.8580985 , 7.851354  , 7.844848  , 7.8312597 , 7.8128505 ,
       7.80223   , 7.7955503 , 7.7853823 , 7.764229  , 7.746432  ,
       7.733869  , 7.730765  , 7.709148  , 7.671208  , 7.645538  ,
       7.619282  , 7.594209  , 7.5739546 , 7.5462565 , 7.525217  ,
       7.5037766 , 7.5005035 , 7.4531174 , 7.4409637 , 7.42580

In [60]:
g.reshape(-1, 8)[:, 0]

array([9.242743  , 9.006223  , 8.714527  , 8.548343  , 8.402171  ,
       8.092784  , 7.8774657 , 7.7955503 , 7.645538  , 7.4531174 ,
       7.3046613 , 7.069288  , 6.897785  , 6.779307  , 6.662062  ,
       6.5349984 , 6.355835  , 6.22721   , 6.132437  , 5.9920955 ,
       5.8473983 , 5.743829  , 5.611202  , 5.52091   , 5.416333  ,
       5.3091044 , 5.1786027 , 5.092963  , 4.9633102 , 4.8552957 ,
       4.749888  , 4.6225023 , 4.5255594 , 4.4632173 , 4.365771  ,
       4.276502  , 4.199949  , 4.1130695 , 4.0380845 , 3.9531665 ,
       3.888375  , 3.8174536 , 3.7590919 , 3.6773427 , 3.5847728 ,
       3.519794  , 3.4285526 , 3.3583326 , 3.3049834 , 3.242573  ,
       3.1863432 , 3.1127894 , 3.046349  , 2.9973443 , 2.899809  ,
       2.8159924 , 2.7515304 , 2.688807  , 2.634713  , 2.5543566 ,
       2.503524  , 2.433517  , 2.3410761 , 2.2860956 , 2.230164  ,
       2.1677105 , 2.0947104 , 2.0294275 , 1.984266  , 1.9200723 ,
       1.8651307 , 1.8040891 , 1.7324016 , 1.6646895 , 1.59875

In [ ]:
import pandas as pd
D = {'x': np.ones(10)}
pd = pd.DataFrame(D).to_json('x.json')